In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

In [ ]:
import pandas as pd

data_frame = pd.read_excel('data_case_study.xlsx')
li = data_frame['Straße'].drop_duplicates().tolist()
addresses = [e + ', Münster' for e in li]

print(addresses)

In [ ]:
import requests

api_file = open("apikey.txt", "r")
api_key = api_file.read()
api_file.close()

url_geocode = "https://maps.googleapis.com/maps/api/geocode/json?"
url_directions = "https://maps.googleapis.com/maps/api/directions/json?"

locations_new = []

for i in addresses:
    response = requests.get(url_geocode + 'address=' + i + '&key=' + api_key)
    dic = response.json()
    lat = dic['results'][0]['geometry']['location']['lat']
    lng = dic['results'][0]['geometry']['location']['lng']
    locations_new.append([lat, lng])

print(locations_new)

In [ ]:
distances = {}
routes = {}
count = 0
total = len(addresses) * (len(addresses) - 1)

for i in range(0, len(addresses)):
    for j in range(i + 1, len(addresses)):
        origin = addresses[i]
        destination = addresses[j]
        response = requests.get(url_directions + "origin=" + origin + "&destination=" + destination+ "&key=" + api_key)
        route = response.json()
        routes[(i, j)] = route
        distance = route["routes"][0]["legs"][0]["distance"]["value"]
        distances[(i, j)] = distance / 1000
        
        routes[(j, i)] = route
        distances[(j, i)] = distance / 1000
        
        '''
        origin = addresses[j]
        destination = addresses[i]
        response = requests.get(url_directions + "origin=" + origin + "&destination=" + destination+ "&key=" + api_key)
        route = response.json()
        routes[(j, i)] = route
        distance = route["routes"][0]["legs"][0]["distance"]["value"]
        distances[(j, i)] = distance / 1000
        '''
        
        count += 2
        print('%3.2f percent complete' %(count / total * 100), end = '\r')
        

In [ ]:
nc = len(addresses) - 1
nr = 0

for i in range(0, nc + nr + 1):
    distances[(i, i)] = 0
    
#print(distances)
#print(routes.keys())

In [ ]:
from tkinter import filedialog
from tkinter import Tk
import os

root = Tk()
path = filedialog.asksaveasfilename(initialdir = os.getcwd(), title = 'Please select a file:')

root.destroy()

print(path)

In [ ]:
north_bound = 51.944711
south_bound = 51.877997
left_bound = 7.551670
right_bound = 7.651555

service_time_customer = 0.1666
service_time_rs = 1
speed = 40
battery_consum_rate = 1
battery_full_level = 200
maximum_time = 6

import pickle
f = open(path,'wb')

pickle.dump(locations_new, f)
pickle.dump(addresses, f)
pickle.dump(distances, f)
pickle.dump(routes, f)

pickle.dump(north_bound, f)
pickle.dump(south_bound, f)
pickle.dump(left_bound, f)
pickle.dump(right_bound, f)

pickle.dump(nc, f)
pickle.dump(nr, f)

pickle.dump(service_time_customer, f)
pickle.dump(service_time_rs, f)
pickle.dump(speed, f)
pickle.dump(battery_consum_rate, f)
pickle.dump(battery_full_level, f)
pickle.dump(maximum_time, f)

f.close()

In [ ]:
lats, lngs = zip(*locations_new)

import gmplot

gmap = gmplot.GoogleMapPlotter((north_bound + south_bound) / 2, (left_bound + right_bound) / 2, 12, apikey = api_key)

gmap.marker(north_bound, left_bound, color = 'black')
gmap.marker(south_bound, left_bound, color = 'black')
gmap.marker(north_bound, right_bound, color = 'black')
gmap.marker(south_bound, right_bound, color = 'black')

gmap.marker(lats[0], lngs[0], color = 'red')

gmap.scatter(lats[1: (nc + 1)], lngs[1: (nc + 1)], color = 'blue', size = 100, marker = False)

for i in range(0, nr):
    
    gmap.marker(lats[1+ nc + i], lngs[1+ nc + i], color = 'gold')

url = 'map_' + path[path.rindex('/') + 1:] + '.html'

gmap.draw(url)

from IPython.display import IFrame

IFrame(url, width = 500, height = 500)